In [1]:
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

from datetime import datetime, timedelta, date, time
import dateutil
from dateutil.parser import parse
import calendar
import numpy as np
from circle_cal.model import CalendarElement
import circle_cal as cc
import circle_cal.model as model
import circle_cal.plot as ccplot
import pathlib
from rich import print
import pytz
from pytz import timezone
import workalendar

In [2]:
cred = pathlib.Path('/Users/kdavis10/.config/.credentials')
cred_json = "pygooglecal.apps.googleusercontent.com.json"

try:
    gcal = GoogleCalendar(credentials_path=cred / cred_json)
except Exception as e:
    (cred / "token.pickle").unlink()
    print("Token expired, re-run this cell.")

In [3]:
year = y24 = model.Year(2024)

traces = []

clist = list(gcal.get_calendar_list())
sel_cal_summ = ["http://universerevealed.nd.edu/?eme_ical=public",
                "NYT Astronomy and Space Calendar",
                "kdavis10@nd.edu",
                "ND-DVT Calendar",
                "CoS College-wide Events & Conferences",
                "Keith Davis",
                "Axis Ludi Calendar",
                "Holidays in United States",
                "Academic Calendar",
                "Seasons"]

selcal = [c for c in clist if c.summary in sel_cal_summ]
print(selcal)

ws = year.weekends()

[
    <CalendarListEntry None - (NYT Astronomy and Space Calendar)>,
    <CalendarListEntry None - (CoS College-wide Events & Conferences)>,
    <CalendarListEntry None - (Seasons)>,
    <CalendarListEntry None - (http://universerevealed.nd.edu/?eme_ical=public)>,
    <CalendarListEntry None - (Holidays in United States)>,
    <CalendarListEntry None - (Axis Ludi Calendar)>,
    <CalendarListEntry None - (Keith Davis)>,
    <CalendarListEntry None - (ND-DVT Calendar)>,
    <CalendarListEntry ND Academic Calendar - (Academic Calendar)>,
    <CalendarListEntry None - (kdavis10@nd.edu)>
]

In [4]:
df = ccplot.selected_cals_to_dataframe(gcal, selcal, y24)
df

/Users/kdavis10/Documents/PythonProjects/taskbraid/python/circlecal/circle_cal/plot.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs, axis="rows", ignore_index=True)


,Event_obj,duration,mid,start,end,summary,color,calendar_id,calendar,weekday
0,<circle_cal.model.EventWrap object at 0x11fd5a...,1 days 00:00:00,2024-01-01 12:00:00-05:00,2024-01-01 00:00:00-05:00,2024-01-02 00:00:00-05:00,Happy New Year From Science Times,#9fc6e7,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...,NYT Astronomy and Space Calendar,Monday
1,<circle_cal.model.EventWrap object at 0x11fd5a...,2 days 00:00:00,2024-01-04 00:00:00-05:00,2024-01-03 00:00:00-05:00,2024-01-05 00:00:00-05:00,The Quadrantid meteor shower will reach its peak.,#9fc6e7,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...,NYT Astronomy and Space Calendar,Wednesday
2,<circle_cal.model.EventWrap object at 0x11fd5a...,2 days 00:00:00,2024-04-22 00:00:00-04:00,2024-04-21 00:00:00-04:00,2024-04-23 00:00:00-04:00,The Lyrid meteor shower will reach its peak.,#9fc6e7,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...,NYT Astronomy and Space Calendar,Sunday
3,<circle_cal.model.EventWrap object at 0x11fd5a...,2 days 00:00:00,2024-08-12 00:00:00-04:00,2024-08-11 00:00:00-04:00,2024-08-13 00:00:00-04:00,The Perseid meteor shower will reach its peak.,#9fc6e7,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...,NYT Astronomy and Space Calendar,Sunday
4,<circle_cal.model.EventWrap object at 0x11fd5a...,1 days 00:00:00,2024-09-22 12:00:00-04:00,2024-09-22 00:00:00-04:00,2024-09-23 00:00:00-04:00,Autumn is here.,#9fc6e7,nytimes.com_89ai4ijpb733gt28rg21d2c2ek@group.c...,NYT Astronomy and Space Calendar,Sunday
...,...,...,...,...,...,...,...,...,...,...
1190,<circle_cal.model.EventWrap object at 0x1280f3...,0 days 01:00:00,2024-09-12 13:00:00-04:00,2024-09-12 12:30:00-04:00,2024-09-12 13:30:00-04:00,Astro Journal Club,#ffad46,kdavis10@nd.edu,kdavis10@nd.edu,Thursday
1191,<circle_cal.model.EventWrap object at 0x1280f3...,0 days 02:30:00,2024-09-05 17:45:00-04:00,2024-09-05 16:30:00-04:00,2024-09-05 19:00:00-04:00,Save the Date! Academic Year kick-off; All-Col...,#ffad46,kdavis10@nd.edu,kdavis10@nd.edu,Thursday
1192,<circle_cal.model.EventWrap object at 0x1280f3...,0 days 01:30:00,2024-09-20 13:15:00-04:00,2024-09-20 12:30:00-04:00,2024-09-20 14:00:00-04:00,Faculty Meeting- External Review,#ffad46,kdavis10@nd.edu,kdavis10@nd.edu,Friday
1193,<circle_cal.model.EventWrap object at 0x1280f3...,0 days 01:30:00,2024-10-04 13:15:00-04:00,2024-10-04 12:30:00-04:00,2024-10-04 14:00:00-04:00,Faculty Meeting,#ffad46,kdavis10@nd.edu,kdavis10@nd.edu,Friday


In [5]:
ii = pd.IntervalIndex([pd.Interval(pd.Timedelta(days=0), pd.Timedelta(hours=23, minutes=59, seconds=59)),
                       pd.Interval(pd.Timedelta(hours=23, minutes=59, seconds=59), pd.Timedelta(days=1)),
                       pd.Interval(pd.Timedelta(days=1), pd.Timedelta(days=31)),
                       pd.Interval(pd.Timedelta(days=31), pd.Timedelta(days=10000))])
intervalsdf = pd.DataFrame(data=["hours", "day", "weeks", "months"], index=ii, columns=["name"])
intervalsdf["opacity"] = [1, 0.8, 0.7, 0.5]
intervalsdf["rbase"] = list(reversed([0.5, 0.6, 0.7, 0.8]))



In [6]:
df["scale"] = pd.cut(df["duration"], intervalsdf.index)

In [7]:
df["ring"] = intervalsdf["name"].loc[df["scale"]].reset_index(drop=True)
df["opacity"] = intervalsdf["opacity"].loc[df["scale"]].reset_index(drop=True)
df["rbase"] = intervalsdf["rbase"].loc[df["scale"]].reset_index(drop=True)

In [8]:
def calendar_ticks(year):
    tickvals = []
    ticktext = []
    for m in year:
        tickvals.append(year.to_theta(m.start))
        #tickvals.append(ts_to_theta(model.to_timestamp(m.mid),
         #                           model.to_timestamp(year.start.datetime()),
          #                          (year.duration / timedelta(days=1)) / 360))
        #ticktext.append("")
        ticktext.append(m.name)
    return (tickvals, ticktext)
        

##def ring_to_base(ring, intervalsdf):
 #   dr = 1/(len(intervalsdf) + 1)
 #   rvalues = [(r+1) * dr for r in range(len(intervalsdf))]   
 #   d = dict(zip(intervalsdf.name.values, reversed(rvalues)))
 #   return d[ring]
    
year=y24

df["midtheta"] = df.mid.apply(year.to_theta)
df["width"] = df.duration.apply(year.to_theta)
#df["rbase"] = df.ring.apply(lambda x: ring_to_base(x, intervalsdf))
#df["rbase"] = .5
df["r"] = .1 * .97 #1/(len(intervalsdf) + 1) * .97


In [9]:
dring = df
try:
    del f
except NameError:
    pass
f = subplotsfig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}, {'type': 'polar'}]])
f = go.Figure(f)
#f = go.Figure()

for cal in pd.unique(df.calendar):
    df = dring[dring["calendar"] == cal]
    f.add_trace(go.Barpolar(theta=df.midtheta, r=df.r, width=df.width, base=df.rbase,
                            text=df.summary, marker_color=df.color, name=cal, #marker_opacity=df.opacity
                            customdata=df[["start", "end"]],
                            
                           )
               )
df = dring
tickvals, ticktext = calendar_ticks(y24)

f.add_trace(go.Barpolar(theta=[o_theta(model.to_timestamp(w.mid), model.to_timestamp(y24.start.datetime())) for w in ws],
                        #r=len(ws) * [.167], base=.167,
                        r=len(ws) * [.1], base=.4,
                        marker_color="#E1E1E1", marker_opacity=.5,
                        width=[(w.duration / year.duration * 360) for w in ws], name="Weekends",
                       customdata=[(w.start, w.end) for w in ws])
           )

f.update_layout(height=800)
f.update_layout(
    template=None,
    polar = dict(
        radialaxis = dict(tickvals=[intervalsdf.rbase],
                          showticklabels=False, ticks='',
                         ),
        angularaxis = dict(
            rotation = -90,
            showticklabels=True,
            ticks='outside',
            ticktext=ticktext,
            tickvals=tickvals,
        )
    )
)
f.update_traces(hovertemplate="%{text}<br>Start: %{customdata[0]|%Y/%m/%d %H:%M:%S.%L}"
               "<br>End: %{customdata[1]|%Y/%m/%d %H:%M:%S.%L}")
#f.add_trace(go.Sunburst(yd))
f.add_trace(go.Scatterpolar(r=[0, 1], theta=2*[ts_to_theta(model.to_timestamp(datetime.now()),
                                                           model.to_timestamp(y24.start.datetime()),
                                                          366 / 360)], name="Now", marker_color="black", text=["Today"]*2))

for t in f.data:
    f.add_trace(t, 1, 2)

f.update_layout(polar2=dict(angularaxis=f.layout["polar"]["angularaxis"],
                           radialaxis=f.layout["polar"]["radialaxis"]))
f.update_layout(polar2=dict(sector=[90,100]))

f.show()

NameError: name 'o_theta' is not defined

In [12]:
f.layout["polar"]

layout.Polar({
    'angularaxis': {'rotation': -90,
                    'showticklabels': True,
                    'ticks': 'outside',
                    'ticktext': [January, February, March, April, May, June, July,
                                 August, September, October, November, December],
                    'tickvals': [0.0, 30.491803278688526, 59.016393442622956,
                                 89.4672131147541, 118.97540983606558,
                                 149.46721311475412, 178.97540983606558,
                                 209.46721311475412, 239.95901639344265,
                                 269.4672131147541, 299.9590163934426,
                                 329.5081967213115]},
    'domain': {'x': [0.0, 0.45], 'y': [0.0, 1.0]},
    'radialaxis': {'showticklabels': False, 'ticks': '', 'tickvals': [[0.8, 0.7, 0.6, 0.5]]}
})

In [13]:
g = iter(year)

In [14]:
for m in g:
    print(m)

{'year': 2024, 'month': 1, 'type': 'CalendarElement'}

{'year': 2024, 'month': 2, 'type': 'CalendarElement'}

{'year': 2024, 'month': 3, 'type': 'CalendarElement'}

{'year': 2024, 'month': 4, 'type': 'CalendarElement'}

{'year': 2024, 'month': 5, 'type': 'CalendarElement'}

...

{'year': 2024, 'month': 8, 'type': 'CalendarElement'}

{'year': 2024, 'month': 9, 'type': 'CalendarElement'}

{'year': 2024, 'month': 10, 'type': 'CalendarElement'}

{'year': 2024, 'month': 11, 'type': 'CalendarElement'}

{'year': 2024, 'month': 12, 'type': 'CalendarElement'}